In [1]:
import pandas as pd
import numpy as np

In [2]:
people = {
    'first': ['Corey', 'Jane', 'John', 'Chris', np.nan, None, 'NA'], 
    'last': ['Schafer', 'Doe', 'Doe', 'Schafer', np.nan, np.nan, 'Missing'], 
    'email': ['CoreyMSchafer@gmail.com', 'JaneDoe@email.com', 'JohnDoe@email.com', None, np.nan, 'Anonymous@email.com', 'NA'],
    'age': ['33', '55', '63', '36', None, None, 'Missing']
}

In [3]:
df = pd.DataFrame(people)

In [4]:
df

,first,last,email,age
0,Corey,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
3,Chris,Schafer,None,36
4,NaN,NaN,NaN,None
5,None,NaN,Anonymous@email.com,None
6,NA,Missing,NA,Missing


### 今天要來做數據清洗: 
#### 最簡單的步驟: 如果數據有 <font color = 'green'>NaN(not a number) </font>，可以用 <font color = 'yellow'> dropna() </font> 來直接清除。

In [5]:
df.dropna()
# 可以發現 pandas 將含有 NaN 的 rows 都去除了。
# 這是 default 的條件，我們也可以客製化自己的條件。

,first,last,email,age
0,Corey,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
6,NA,Missing,NA,Missing


In [6]:
df.dropna(axis = 'index', how = 'any') # 這是 default 下的參數設置，結果跟不給參數是一樣的。

,first,last,email,age
0,Corey,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
6,NA,Missing,NA,Missing


In [7]:
df.dropna(axis = 'index', how = 'all') 
# axis 是指當要去除時，是去除整個 row [axis = 'index']還是整個 column。[axis = 'columns']
# how 是指刪除的條件
    # all: 當 row(或 column) 所有資料都是 NaN，才去除。
    # any: 當 row(或 column) 任一資料出現 NaN，才去除。

,first,last,email,age
0,Corey,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
3,Chris,Schafer,None,36
5,None,NaN,Anonymous@email.com,None
6,NA,Missing,NA,Missing


### 情境: 清洗數據時，某些 column 有缺資料沒關係。
#### 假設: 我們只須確保 email column 都有數據，其他 columns 有缺沒關係。
#### 這時要多加<font color = 'yellow'>參數 subset(用一個 list 將你要判斷的目標包起來) </font>

In [8]:
df.dropna(axis = 'index', how = 'any', subset = ['email'])

,first,last,email,age
0,Corey,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
5,None,NaN,Anonymous@email.com,None
6,NA,Missing,NA,Missing


#### 如果改成: df.dropna(axis = 'index', how = 'all', subset = ['last', 'email']) 會如何?

In [9]:
df.dropna(axis = 'index', how = 'all', subset = ['last', 'email'])
# 當 last 跟 email 都是 NaN 時，才會將那個 row 去掉。

,first,last,email,age
0,Corey,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
3,Chris,Schafer,None,36
5,None,NaN,Anonymous@email.com,None
6,NA,Missing,NA,Missing


### 在第 6 row，NA 跟 Missing 是直接用 string 表示的，我們想要換成 numpy 的 NaN。

In [10]:
df.replace('NA', np.nan, inplace = True)
df.replace('Missing', np.nan, inplace = True)

In [11]:
df.isna()
# isna() 顯示資料中哪些是真的 NaN。

,first,last,email,age
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,True,False
4,True,True,True,True
5,True,True,False,True
6,True,True,True,True


In [12]:
df.fillna(0)
# 將所有 NaN 改為數字 0。
# 情境: 如果學生缺少成績(NaN)，就當 0 分計算(數字 0)。

,first,last,email,age
0,Corey,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
3,Chris,Schafer,0,36
4,0,0,0,0
5,0,0,Anonymous@email.com,0
6,0,0,0,0


In [13]:
df.dtypes # 查看 columns 的屬性

first    object
last     object
email    object
age      object
dtype: object

In [15]:
df['age'].mean()
# 由於 age 是 string 的形式，所以不能計算平均。

TypeError: can only concatenate str (not "int") to str

In [16]:
# 但要注意的是，轉換時如果有 NaN，則要轉成 float(非 int)
type(np.nan)
# 因為 NaN 本身就是 float

float

In [17]:
df['age'] = df['age'].astype(float)

In [18]:
# 這樣就可以計算平均了
df['age'].mean()

46.75

## 我們現在用真實世界 csv 檔為例子:
### 用關鍵字 <font color = 'yellow'> na_values </font> 讓 csv 檔在讀入時就將 NA 或 Missing 字串轉為 NaN。

In [21]:
na_vals = ['NA', 'Missing']

df = pd.read_csv('data/survey_results_public.csv', index_col='Respondent', na_values = na_vals)
schema_df = pd.read_csv('data/survey_results_schema.csv', index_col='Column')

In [20]:
pd.set_option('display.max_columns', 85)
pd.set_option('display.max_rows', 85)

### 任務: 計算 YearsCode column 的平均:

In [22]:
df['YearsCode'].mean()
# 直接先試 .mean()，發現有錯

TypeError: can only concatenate str (not "int") to str

In [25]:
df.dtypes
# 發現 YearsCode 是物件，因此我們想轉換成 float(畢竟可能有 NaN)

MainBranch                 object
Hobbyist                   object
OpenSourcer                object
OpenSource                 object
Employment                 object
Country                    object
Student                    object
EdLevel                    object
UndergradMajor             object
EduOther                   object
OrgSize                    object
DevType                    object
YearsCode                  object
Age1stCode                 object
YearsCodePro               object
CareerSat                  object
JobSat                     object
MgrIdiot                   object
MgrMoney                   object
MgrWant                    object
JobSeek                    object
LastHireDate               object
LastInt                    object
FizzBuzz                   object
JobFactors                 object
ResumeUpdate               object
CurrencySymbol             object
CurrencyDesc               object
CompTotal                 float64
CompFreq      

In [26]:
df['YearsCode'] = df['YearsCode'].astype(float)
# 又發現錯誤，因為有一個 'Less than 1 year' 無法被轉換成 flaot。

ValueError: could not convert string to float: 'Less than 1 year'

In [27]:
# 我們先看一下所有選項長甚麼樣子，可以用 value_counts()，也可以:
df['YearsCode'].unique()

array(['4', nan, '3', '16', '13', '6', '8', '12', '2', '5', '17', '10',
       '14', '35', '7', 'Less than 1 year', '30', '9', '26', '40', '19',
       '15', '20', '28', '25', '1', '22', '11', '33', '50', '41', '18',
       '34', '24', '23', '42', '27', '21', '36', '32', '39', '38', '31',
       '37', 'More than 50 years', '29', '44', '45', '48', '46', '43',
       '47', '49'], dtype=object)

In [29]:
# 原來阻饒我們的有: Less than 1 year 跟 More than 50 years，因此:
df['YearsCode'].replace('Less than 1 year', 0, inplace = True)
df['YearsCode'].replace('More than 50 years', 51, inplace = True)

In [31]:
# 再使用 astype() 轉換成 float:
df['YearsCode'] = df['YearsCode'].astype(float)

In [33]:
# 這樣就可以計算相關統計量了:
df['YearsCode'].mean()

11.662114216834588

In [34]:
df['YearsCode'].median()

9.0